In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import layers
from keras import models
%matplotlib inline

In [15]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [16]:
train_data, validation_data = train_test_split(train_data, test_size = 0.1, shuffle=True)

x_train = (train_data.iloc[:,1:].values).astype('float32').reshape((-1, 28, 28, 1))
y_train = np.array(pd.get_dummies(train_data.iloc[:,0])).astype('float32')
x_val = validation_data.iloc[:, 1:].values.astype('float32').reshape((-1, 28, 28, 1))
y_val = np.array(pd.get_dummies(validation_data.iloc[:,0])).astype('float32')
x_test = test_data.iloc[:, :].values.astype('float32').reshape(-1, 28, 28, 1)

In [23]:
x_train /= 255
x_val /= 255
x_test /= 255

In [17]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10, activation='softmax'))

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
batch_normalization_6 (Batch (None, 11, 11, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_3 (Flatten)          (None, 576)               0         
__________

In [19]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
from keras.preprocessing import image

In [26]:
datagen = image.ImageDataGenerator(rotation_range=40, 
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2, 
                                    shear_range=0.2, 
                                    zoom_range=0.2, 
                                    horizontal_flip=True)

In [27]:
datagen.fit(x_train)

In [32]:
batch_size = 100
epochs = 5

In [34]:
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_val, y_val),
                              verbose = 2, steps_per_epoch=x_train.shape[0] // batch_size)

Epoch 1/5
 - 41s - loss: 1.4699 - acc: 0.4990 - val_loss: 0.4874 - val_acc: 0.8388
Epoch 2/5
 - 39s - loss: 0.8063 - acc: 0.7291 - val_loss: 0.2865 - val_acc: 0.9195
Epoch 3/5
 - 38s - loss: 0.6141 - acc: 0.7998 - val_loss: 0.2869 - val_acc: 0.9083
Epoch 4/5
 - 38s - loss: 0.5065 - acc: 0.8360 - val_loss: 0.4997 - val_acc: 0.8643
Epoch 5/5
 - 37s - loss: 0.4542 - acc: 0.8552 - val_loss: 0.1575 - val_acc: 0.9552


In [35]:
model.evaluate(x_val, y_val)

4200/4200 [==============================] - 2s 410us/step


[0.15745610013958955, 0.9552380952380952]

In [61]:
predications = np.argmax(model.predict(x_test), 1)
result = pd.DataFrame({'ImageId': np.arange(len(predications)) + 1, 'Label': predications})
result.to_csv('submission.csv', index=False)

In [62]:
!kaggle competitions submit -c digit-recognizer -f submission.csv -m "Message"

Successfully submitted to Digit Recognizer